In [1]:
# !pip install spacy
# !pip install keypartx[coreferee_spacy]
# !python3 -m coreferee install en 
# !python -m spacy download en_core_web_lg
# !pip install keypartx
# # !python -m spacy download en_core_web_lg
# !pip install git+https://github.com/pengKiina/pyvis.git

In [2]:
import os
from llama_index.query_engine import CitationQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    LLMPredictor,
    ServiceContext,
)


In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import umap, hdbscan
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 

# no need for clean data which has removed stopwords and number, stemmatized extra...
# my_list_of_stopwords = text.ENGLISH_STOP_WORDS.union(stopwords_fi +stopwords_sv)
#vectorizer_model_multi = CountVectorizer(stop_words=my_list_of_stopwords, ngram_range=(1, 2), min_df=2) # finnish + english 
#vectorizer_model = CountVectorizer(stop_words=my_list_of_stopwords, ngram_range=(1,2), min_df=2) #The min_df parameter is used to indicate the minimum frequency of words. Setting this value larger than 1 can significantly reduce memory
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1,2), min_df=2) #The min_df parameter is used to indicate the minimum frequency of words. Setting this value larger than 1 can significantly reduce memory


# Load sentence transformer model
# 'sentence-transformers/distiluse-base-multilingual-cased-v2'
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")# english default "all-MiniLM-L6-v2"   
#embedding_model= pipeline('fill-mask', model='Finnish-NLP/roberta-large-finnish')
#embedding_model_multi = pipeline('fill-mask', model='bert-base-multilingual-cased') # multilingual bert
embedding_model_multi = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # multingual ST


# Create documents embeddings
#embeddings = embedding_model.encode(docs, show_progress_bar=False)

# Define UMAP model to reduce embeddings dimension
umap_model = umap.UMAP(n_neighbors=15,
                        n_components=1,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False 
                       ) #, low_memory=False # to reduce topic numbers, UMAP model and set n_neighbors much higher than the default 15 (e.g., 200). 

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True) # the bigger cluster size for bigger documents 


/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/joramas/miniconda/envs/thesis/lib/python3.9/site-packages/umap/d

In [4]:
from keypartx.basemodes.avn_base import lemma_en
from sklearn.feature_extraction.text import TfidfVectorizer

No Coreference package avaiable


In [5]:
# import pandas as pd
# text_link = "https://raw.githubusercontent.com/pengKiina/KeypartX/main/data/Thailand_text.csv"
# commentsDF = pd.read_csv(text_link)
# import re
# comments_en = commentsDF.review.to_list()
# cleaned_text = []
# for comment in comments_en:
#     comment_nonStop =  " ".join([word for word in comment.split() if len(word)>3])
#     comment_lemma = lemma_en(comment_nonStop)
#     comment_nonum = re.sub(r'[0-9]+',' ', comment_lemma)
#     cleaned_text.append(comment_nonum)

In [6]:
# Create BERTopic model EN
# topic_model_en = BERTopic(top_n_words=10,
#                        calculate_probabilities=False,
#                        umap_model= umap_model,
#                        hdbscan_model=hdbscan_model,
#                        vectorizer_model=vectorizer_model,
#                        embedding_model= embedding_model_multi,
#                        verbose=True)
# #topic_model_en = BERTopic()
# topic_model_en.max_seq_length = 510


# review_train = cleaned_text

# # Train model, extract topics and probabilities
# topics_en, probabilities_en = topic_model_en.fit_transform(review_train)
# #topics, _ = topic_model.fit_transform(reviews_en)
# topic_terms = pd.DataFrame(topic_model_en.get_topics())
# topic_terms

In [7]:
# from sentence_transformers.cross_encoder import CrossEncoder
# model = CrossEncoder('https://huggingface.co/cross-encoder/quora-roberta-large')
# scores = model.predict([('Question 1', 'Question 2'), ('Question 3', 'Question 4')])

In [8]:

#Load the model
# model = Model(model_path="gpt4all-lora-quantized.bin", n_ctx=2000)

#Generate
# prompt="User: How are you doing?\nBot:"

# result=model.generate(prompt,n_predict=50)


In [9]:
from langchain.llms import gpt4all
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.llm import LangchainLLM
# load language model
# language_model = gpt4all.GPT4All(model="./llama-2-7b-chat.ggmlv3.q4_0.bin")
# language_model = gpt4all.GPT4All(model="./llama-2-7b-chat.ggmlv3.q4_0.bin")
language_model = LangchainLLM( llm=gpt4all.GPT4All(model="./ggml-model-gpt4all-falcon-q4_0.bin"))
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

service_context = ServiceContext.from_defaults(
    llm=language_model,
    embed_model = embed_model,
    chunk_size=100
)                    

ModuleNotFoundError: No module named 'llama_index.llm'

In [ ]:
# https://qdrant.tech/documentation/integrations/llama-index/
if not os.path.exists("./citation"):
    documents = SimpleDirectoryReader("documents/paul_graham").load_data()
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    index.storage_context.persist(persist_dir="./citation")
else:
    index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./citation"),
        service_context=service_context,
    )

[2023-08-03 19:03:47,109] {loading.py:63} INFO - Loading all indices.


In [ ]:
# query_engine = index.as_query_engine(streaming=True)

# streaming_response = query_engine.query("Who is Paul Graham.")
# streaming_response.print_response_stream() 

In [ ]:
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=52,
)

In [ ]:
response = query_engine.query("summary")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(response)
# source nodes are 6, because the original chunks of 1024-sized nodes were broken into more granular nodes
print(len(response.source_nodes))

ERROR: The prompt size exceeds the context window size and cannot be processed.
2
